# Introduction

***Notebook in progress. Defining classes of Player and Game to prep for simulations.***

## Imports

In [38]:
import numpy as np
np.random.seed(0)
import operator
import pandas as pd

# Helper Function

Creating a function that randomly rolls the LCR die `num_dice` number of times and returns the total roll as a list.

In [2]:
def roll_LCR(num_dice):
    '''
    Rolls n dice and returns the output as a list of n strings.
    '''
    output = []
    for i in range(num_dice):
        roll = np.random.choice(["L", "C", "R", "O", "O", "O"])
        output.append(roll)
    return output

In [3]:
# Example
roll_LCR(8)

['O', 'O', 'L', 'O', 'O', 'O', 'C', 'O']

# `Player` Class

In [4]:
class Player:
    
    def __init__(self, player_num, coins = 0, to_left = None,
                 to_right = None, game = None):
        self.player_num = int(player_num)
        self.coins = int(coins)
        self.to_left = to_left
        self.to_right = to_right
        self.game = game
        
    def roll(self, num_dice):
        '''
        Rolls n dice and returns the output as a list of n strings. Wrapper.
        '''
        roll = roll_LCR(num_dice)
        return roll

    def pass_coins(self, roll):
        '''
        Takes in a roll as list from roll() function. Passes coins to
        appropriate players and pot. Subtracts lost coins from self.
        '''
# Subtract coins from self
        self.coins -= len([x for x in roll if x != 'O'])
# Pass coins to players on right and left
        self.to_right.coins += len([x for x in roll if x == 'R'])
        self.to_left.coins += len([x for x in roll if x == 'L'])
# Pass coins to central pot via game class
        self.game.central_pot += len([x for x in roll if x == 'C'])

# `Game` Class

In [5]:
class Game:
    
    def __init__(self, current_round = 0, central_pot = 0):
        self.current_round = current_round
        self.central_pot = central_pot

# Example

Below we can see how the game is set up, and how it functions. By passing the instances of the `Player` class to each other, we can make the game work *relationally.* They all are able to update each other's coins, and passing them becomes extremely simple.

In [11]:
# Create a game
game = Game()

# Create players
emi = Player(player_num = 1, coins = 3, to_left = None,
             to_right = None, game = game)

steven = Player(player_num = 2, coins = 3, to_left = emi,
                to_right = None, game = game)

matt = Player(player_num = 3, coins = 3, to_left = steven,
              to_right = emi, game = game)

# Some backtracking is needed to set up relationships
emi.to_left, emi.to_right = matt, steven
steven.to_right = matt

The cell above is effectively our "prep" section. The section below is where the game is actually "played."

In [12]:
# Have player Emi roll three dice
her_roll = emi.roll(3)

# She will put two in central pot and pass one to player on left
her_roll

['C', 'L', 'C']

In [13]:
# She has three coins before passing

emi.coins

3

In [14]:
# All coins are passed
emi.pass_coins(her_roll)

# We see she has zero, Matt gained one, and the central pot gained two
print(emi.coins, steven.coins, matt.coins, game.central_pot)

0 3 4 2


Here's another example where Matt passes one of his coins to the central pot. The game continues like this until only one player and the pot have coins.

In [17]:
matts_roll = matt.roll(3)
matts_roll

['O', 'C', 'O']

In [18]:
matt.pass_coins(matts_roll)
print(emi.coins, steven.coins, matt.coins, game.central_pot)

0 3 3 3


# Scratch paper

How to determine the game's winner:

In [44]:
players = [emi, steven, matt]

sorted_x = sorted(players, key = operator.attrgetter('coins'), reverse=True)

In [47]:
print(emi.coins, steven.coins, matt.coins, game.central_pot)
print("The winner is player", sorted_x[0].player_num)

0 3 9 3
The winner is player 3
